Steel faults.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

## Imports

In [3]:
import numpy as np
from matplotlib import pyplot as plt
import scipy as sp
import pandas as pd

from sklearn import datasets 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

## Load data

In [4]:
df = pd.read_csv('../data/steel_plates_faults.csv')

df.head()

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Fault
0,42,50,270900,270944,267,17,44,24220,76,108,...,0.4706,1.0000,1.0,2.4265,0.9031,1.6435,0.8182,-0.2913,0.5822,Pastry
1,645,651,2538079,2538108,108,10,30,11397,84,123,...,0.6000,0.9667,1.0,2.0334,0.7782,1.4624,0.7931,-0.1756,0.2984,Pastry
2,829,835,1553913,1553931,71,8,19,7972,99,125,...,0.7500,0.9474,1.0,1.8513,0.7782,1.2553,0.6667,-0.1228,0.2150,Pastry
3,853,860,369370,369415,176,13,45,18996,99,126,...,0.5385,1.0000,1.0,2.2455,0.8451,1.6532,0.8444,-0.1568,0.5212,Pastry
4,1289,1306,498078,498335,2409,60,260,246930,37,126,...,0.2833,0.9885,1.0,3.3818,1.2305,2.4099,0.9338,-0.1992,1.0000,Pastry


In [5]:
feature_names = list(set(df.columns) - set(['Fault']))

feature_names

['X_Maximum',
 'TypeOfSteel_A400',
 'Pixels_Areas',
 'Empty_Index',
 'Edges_Index',
 'Log_X_Index',
 'Luminosity_Index',
 'Sum_of_Luminosity',
 'TypeOfSteel_A300',
 'LogOfAreas',
 'Orientation_Index',
 'Steel_Plate_Thickness',
 'Square_Index',
 'Maximum_of_Luminosity',
 'Outside_Global_Index',
 'X_Perimeter',
 'Edges_X_Index',
 'Y_Maximum',
 'Y_Minimum',
 'Log_Y_Index',
 'Minimum_of_Luminosity',
 'X_Minimum',
 'Y_Perimeter',
 'Length_of_Conveyer',
 'SigmoidOfAreas',
 'Outside_X_Index',
 'Edges_Y_Index']

In [6]:
X = df[feature_names]
y = df['Fault']

X

,X_Maximum,TypeOfSteel_A400,Pixels_Areas,Empty_Index,Edges_Index,Log_X_Index,Luminosity_Index,Sum_of_Luminosity,TypeOfSteel_A300,LogOfAreas,...,Y_Maximum,Y_Minimum,Log_Y_Index,Minimum_of_Luminosity,X_Minimum,Y_Perimeter,Length_of_Conveyer,SigmoidOfAreas,Outside_X_Index,Edges_Y_Index
0,50,False,267,0.2415,0.0498,0.9031,-0.2913,24220,True,2.4265,...,270944,270900,1.6435,76,42,44,1687,0.5822,0.0047,1.0000
1,651,False,108,0.3793,0.7647,0.7782,-0.1756,11397,True,2.0334,...,2538108,2538079,1.4624,84,645,30,1687,0.2984,0.0036,0.9667
2,835,False,71,0.3426,0.9710,0.7782,-0.1228,7972,True,1.8513,...,1553931,1553913,1.2553,99,829,19,1623,0.2150,0.0037,0.9474
3,860,True,176,0.4413,0.7287,0.8451,-0.1568,18996,False,2.2455,...,369415,369370,1.6532,99,853,45,1353,0.5212,0.0052,1.0000
4,1306,True,2409,0.4486,0.0695,1.2305,-0.1992,246930,False,3.3818,...,498335,498078,2.4099,37,1289,260,1353,1.0000,0.0126,0.9885
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1936,277,True,273,0.3906,0.3662,1.4472,0.0026,35033,False,2.4362,...,325796,325780,1.2041,119,249,22,1360,0.7254,0.0206,0.7273
1937,175,True,287,0.4554,0.2118,1.4914,-0.0582,34599,False,2.4579,...,340598,340581,1.2305,112,144,24,1360,0.8173,0.0228,0.7083
1938,174,True,292,0.3287,0.2132,1.4624,0.0052,37572,False,2.4654,...,386794,386779,1.1761,120,145,22,1360,0.7079,0.0213,0.6818
1939,170,True,419,0.5904,0.2015,1.5185,-0.0171,52715,False,2.6222,...,422528,422497,1.4914,117,137,47,1360,0.9919,0.0243,0.6596


In [7]:
for f_name in feature_names:
    
    if df[f_name].dtype == bool:
        print(f_name)

TypeOfSteel_A400
TypeOfSteel_A300


In [8]:
df[['TypeOfSteel_A300', 'TypeOfSteel_A400']]

,TypeOfSteel_A300,TypeOfSteel_A400
0,True,False
1,True,False
2,True,False
3,False,True
4,False,True
...,...,...
1936,False,True
1937,False,True
1938,False,True
1939,False,True


In [9]:
np.all(df['TypeOfSteel_A300'] == (~df['TypeOfSteel_A400']))

True

In [10]:
scaler = StandardScaler()
_ = scaler.fit(X)

In [11]:
X_scaled = scaler.transform(X)

## Split data

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=42)

In [13]:
X_train.shape

(1455, 27)

In [14]:
X_test.shape

(486, 27)

## Train estimators

In [15]:
estimators = {
    'decision_tree': DecisionTreeClassifier(random_state=24),
    'random_forest': RandomForestClassifier(random_state=24),
    'adaboost': AdaBoostClassifier(random_state=24)
}

In [16]:
n_total = len(y_test)

for k, estimator in estimators.items():

    estimator.fit(X_train, y_train)
    
    y_pred = estimator.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    
    n_matches = np.sum(y_test == y_pred)
    
    print(k)
    print('Number of matches: {} out of {} (accuracy = {:.3f})'.format(n_matches, n_total, acc))
    print()
    
    

decision_tree
Number of matches: 361 out of 486 (accuracy = 0.743)

random_forest
Number of matches: 368 out of 486 (accuracy = 0.757)

adaboost
Number of matches: 273 out of 486 (accuracy = 0.562)

